In [ ]:
import sys 
sys.path.append('..')
from utils.paths import make_dirs, CLEANED_DIR, POTATOES_HASHTAGS_CLEAN, MEANINGLESS_HASHTAGS_LIST, SEARCH_HASHTAGS_LIST, QUARTER_RESULTS, POTATOES_CLEANED_DIR
from utils.utils import load_dataframes, load_dataframe, write_dict_to_csv, write_list_to_csv, write_df_to_csv
import pandas as pd
import os

make_dirs()

def listsOverlap(rowHashtags, hashtags):
    if len(set.intersection(set(rowHashtags), set(hashtags))) > 0:
        return True
    else:
        return False

### Clean downloaded posts
- merge data from different hashtags (if used) for the same quarter
- remove posts that are not related to Munich for quarters with ambiguous names
- remove uninformative hashtags
- remove irrelevant columns from data

In [ ]:
df_hashtags_clean = POTATOES_HASHTAGS_CLEAN()
selected_columns = ['owner_id','shortcode','hashtags', 'post_url', 'is_video', 'likes']
df_post_counts = pd.DataFrame(columns=['quarter', 'post_count', 'post_count_cleaned'])

meaningless_hashtags = set(MEANINGLESS_HASHTAGS_LIST())
search_hashtags = SEARCH_HASHTAGS_LIST()

for index, row in df_hashtags_clean.iterrows():
    # merge data from different hashtags (if used) for the same quarter
    print('>> now processing', row.POTATOES)
    if len(row.HASHTAG)>0:
        df = pd.DataFrame()
        for hashtag in row.HASHTAG:
            df_new = QUARTER_RESULTS(hashtag)
            df_new_hashtags = df_new[selected_columns]
            df = df.append(df_new_hashtags,ignore_index=True)
    else:
        df = QUARTER_RESULTS(row.POTATOES)
        df =df[selected_columns]
    
    # Remove posts that are not related to Munich for quarters with ambiguous names
    post_count = len(df)
    if (len(row.CLEAN)>0):
        mask = (df['hashtags'].apply(lambda x: listsOverlap(x, search_hashtags)))
        df = df[mask]
        
    post_count_cleaned = len(df)
    new_row = {'quarter': row.POTATOES, 'post_count': post_count, 'post_count_cleaned': post_count_cleaned}
    df_post_counts = df_post_counts.append(new_row, ignore_index=True)

    # remove uninformative hashtags
    df['hashtags'] = df['hashtags'].apply(lambda x: sorted(set(x).difference(meaningless_hashtags)))

    # write files
    write_df_to_csv(df, row.POTATOES, POTATOES_CLEANED_DIR)
    
write_df_to_csv(df_post_counts, 'post_counts', CLEANED_DIR)